In [1]:
pip install torch

  Using cached torch-1.13.0-cp310-none-macosx_11_0_arm64.whl (55.7 MB)
  Using cached typing_extensions-4.4.0-py3-none-any.whl (26 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
from torch import nn
from torch.nn import functional as F
# 定义层
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10)) # nn.Sequential定义了一种特殊的module
X = torch.rand(2, 20)
net(X)

tensor([[-0.0699,  0.2026, -0.2308,  0.0451, -0.2832,  0.2267,  0.0852, -0.0178,
         -0.0105, -0.1372],
        [-0.0452,  0.1692, -0.2843,  0.0721, -0.1893,  0.2991,  0.0411,  0.0022,
         -0.1477, -0.2305]], grad_fn=<AddmmBackward0>)

In [5]:
# 定义块
class MLP(nn.Module):
    def __init__(self): # 需要哪些参数
        super().__init__()
        self.hidden = nn.Linear(20, 256) # 隐藏层
        self.out = nn.Linear(256, 10) # 输出层
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [6]:
net = MLP()
net(X)

tensor([[-0.2298,  0.2844, -0.1888,  0.1566, -0.0434, -0.1462,  0.1935,  0.0622,
          0.0583, -0.0281],
        [-0.2088,  0.3343, -0.2733,  0.1026, -0.1271, -0.2454,  0.2193,  0.1003,
          0.0239, -0.1739]], grad_fn=<AddmmBackward0>)

In [8]:
# 定义顺序块
class MySequential(nn.Module):
    def __init__(self, *args): # *args可以在后面执行传入层的操作
        super().__init__()
        for block in args:
            self._modules[block] = block
            
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.0509,  0.0957, -0.0137,  0.0638, -0.0475, -0.0427, -0.2248,  0.1739,
          0.1215, -0.0422],
        [-0.2069,  0.0816, -0.1178,  0.0055, -0.0087,  0.0027, -0.2372,  0.1213,
          0.0053, -0.0476]], grad_fn=<AddmmBackward0>)

In [10]:
# 在正向传播函数中执行代码
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
        
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
net = FixedHiddenMLP()
net(X)

tensor(0.0817, grad_fn=<SumBackward0>)

In [12]:
# 混合搭配各种组合块
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
        
    def forward(self, X):
        return self.linear(self.net(X))
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.0327, grad_fn=<SumBackward0>)

In [13]:
 # 参数管理

In [14]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[0.2481],
        [0.2240]], grad_fn=<AddmmBackward0>)

In [15]:
 print(net[2].state_dict()) # 把每一层中的权重拿出来（参数访问） 

OrderedDict([('weight', tensor([[ 0.1478, -0.0269, -0.0340,  0.2726, -0.1104, -0.1144,  0.1362, -0.0860]])), ('bias', tensor([0.2504]))])


In [16]:
# 直接访问具体参数
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([0.2504], requires_grad=True)
tensor([0.2504])


In [17]:
net[2].weight.grad == None # 因为目前还没有计算梯度所以为none（反向传播）

True

In [18]:
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()]) # 一次性访问所有参数

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [19]:
 net.state_dict()['2.bias'].data

tensor([0.2504])

In [20]:
# 从嵌套块里收集参数
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4),
                        nn.ReLU())
def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block{i}', block1())
        return net
rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.3799],
        [0.3777]], grad_fn=<AddmmBackward0>)

In [21]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [22]:
# 初始化参数（内置初始化）
def init_normal(m): # 传入一个module nomal--正态分布
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)
net.apply(init_normal)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0064,  0.0010, -0.0162,  0.0072]), tensor(0.))

In [26]:
def init_constant(m):  # 初始成一个constant
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [29]:
# 对某些块应用不同的初始化方法
def xavier(m):  
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight) # xavier方法均匀分布
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42) # 固定为42

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([ 0.5130,  0.1763,  0.4363, -0.6071])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [30]:
# 参数绑定（在一些层中共享参数）
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared,
                   nn.ReLU(), nn.Linear(8, 1))
net(X)
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


In [33]:
# 构造没有参数的自定义层
import torch
import torch.nn.functional as F
from torch import nn

class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
    def forward(self, X):
        return X - X.mean()

layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [34]:
# 将层作为组件合并到更复杂的模型中
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

Y = net(torch.rand(4, 8))
Y.mean()

tensor(-1.8626e-09, grad_fn=<MeanBackward0>)

In [35]:
# 带参数的图层
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
        
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)
    
dense = MyLinear(5, 3)
dense.weight

Parameter containing:
tensor([[-0.1951,  0.4563, -0.7224],
        [ 1.9813,  0.7112,  1.4009],
        [ 0.5344,  1.0267,  1.0233],
        [ 0.3169,  0.0659, -0.9727],
        [-1.3295,  2.0167, -0.6337]], requires_grad=True)

In [36]:
dense(torch.rand(2, 5))

tensor([[0.2720, 0.6780, 1.4522],
        [0.0000, 1.3793, 0.0000]])

In [37]:
# 自定义层构建模型
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[1.9988],
        [9.6868]])

In [38]:
# 读写文件

In [39]:
# 加载和保存张量
x = torch.arange(4)
torch.save(x, 'x-file')
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [40]:
# 存储张量列表，读回内存
y = torch.zeros(4)
torch.save([x, y], 'x-files')
x2, y2 =torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [41]:
# 写入和读取从字符串映射到张量的字典
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

In [43]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)
    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size = (2, 20))
y = net(X)

In [44]:
torch.save(net.state_dict(), 'mlp.params')Y_

In [45]:
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [51]:
Y_clone = clone(X)